In [1]:
import numpy as np
import pandas as pd
import sys
sys.path.append("..")
#from utils import get_dvc_dir

In [2]:
#features_dir = get_dvc_dir("features")

In [3]:
pwd

'/Users/evautz/Documents/NASAharvest/crop-mask/notebooks'

In [4]:
tigray21 = pd.read_csv('../data/processed/Ethiopia_Tigray_2021.csv')
# tigray.head()

In [5]:
tigray_halfprob = tigray21.loc[tigray21['crop_probability'] == 0.5]

In [40]:
# CLEANING IT UP:

x = tigray21['num_labelers'].max()
users = [None]*x
for i in range(x):
    num = i+1
    users[i] = f'User {num}'
    
durations = [None]*x
for i in range(x):
    num = i+1
    durations[i] = f'User {num} duration'

tigray21[users] =tigray21['email'].str.split(',',expand=True)
tigray_halfprob[users] = tigray_halfprob['email'].str.split(',',expand=True)
tigray21[durations] = tigray21['analysis_duration'].str.split(',',expand=True)

In [7]:
# Other ways to analyze users
# fewsnet versus gmail vs umd
# subject matter expert (cnakolembe and hkerner (proxy experts))

# For each user, what percent of their labels end up being disagreed points?

In [8]:
tigray21['User 1'].value_counts()

logdaye@gmail.com     672
alopez11@umd.edu      107
hkerner@umd.edu        61
bweeks@fews.net        36
izvonkov@umd.edu        9
bdubayah@gmail.com      1
Name: User 1, dtype: int64

In [9]:
tigray21['User 1'].unique()

array(['bweeks@fews.net', 'izvonkov@umd.edu', 'alopez11@umd.edu',
       'hkerner@umd.edu', 'logdaye@gmail.com', 'bdubayah@gmail.com'],
      dtype=object)

In [10]:
labelers = np.array([])
for i in range(1,7):
    labelers = np.append(labelers, tigray21[f'User {i}'].unique())
labelers

array(['bweeks@fews.net', 'izvonkov@umd.edu', 'alopez11@umd.edu',
       'hkerner@umd.edu', 'logdaye@gmail.com', 'bdubayah@gmail.com',
       'bdubayah@gmail.com', 'logdaye@gmail.com', 'hkerner@umd.edu',
       'alopez11@umd.edu', 'mmitkish@umd.edu', 'cnakalem@umd.edu',
       'eutzschn@umd.edu', None, 'meredith.g.brown@nasa.gov',
       'sophiaowens12@gmail.com', 'daniel.p.sarmiento@nasa.gov',
       'oboiko@contractor.usgs.gov', 'seth.h.peterson@gmail.com', None,
       'bdubayah@gmail.com', 'logdaye@gmail.com', 'cnakalem@umd.edu',
       'mmitkish@umd.edu', 'alopez11@umd.edu', 'sophiaowens12@gmail.com',
       'eutzschn@umd.edu', 'seth.h.peterson@gmail.com',
       'oboiko@contractor.usgs.gov', 'meredith.g.brown@nasa.gov',
       'daniel.p.sarmiento@nasa.gov', None, 'sophiaowens12@gmail.com',
       'eutzschn@umd.edu', 'seth.h.peterson@gmail.com',
       'bdubayah@gmail.com', 'oboiko@contractor.usgs.gov',
       'daniel.p.sarmiento@nasa.gov', 'mmitkish@umd.edu',
       'meredith.g.b

In [11]:
uniqueNames = []
for user in labelers:
    if user not in uniqueNames:
        uniqueNames.append(user)

uniqueNames.remove(None)    
uniqueNames

['bweeks@fews.net',
 'izvonkov@umd.edu',
 'alopez11@umd.edu',
 'hkerner@umd.edu',
 'logdaye@gmail.com',
 'bdubayah@gmail.com',
 'mmitkish@umd.edu',
 'cnakalem@umd.edu',
 'eutzschn@umd.edu',
 'meredith.g.brown@nasa.gov',
 'sophiaowens12@gmail.com',
 'daniel.p.sarmiento@nasa.gov',
 'oboiko@contractor.usgs.gov',
 'seth.h.peterson@gmail.com']

In [12]:
# Want to find count of every bweeks in the table

userDf = pd.DataFrame(data={'user':uniqueNames})
userDf = userDf.set_index('user')

In [33]:
# Find total number of labels each user in this set completed

user1 = (tigray21[f'User 1'].value_counts()).to_frame()
namesDf = user1.merge(userDf, left_index=True,right_index=True)

for i in range(2,7):
    useri = (tigray21[f'User {i}'].value_counts()).to_frame()
    namesDf = namesDf.merge(useri, how='outer',left_index=True,right_index=True)
    
namesDf = namesDf.replace(np.nan, 0)
namesDf = namesDf.astype(int)

namesDf['total_labels'] = 0  
for i in range(1,7):
    namesDf['total_labels'] += namesDf[f'User {i}']
    
namesDf    

,User 1,User 2,User 3,User 4,User 5,User 6,total_labels
alopez11@umd.edu,107,40,1,0,0,0,148
bdubayah@gmail.com,1,29,26,3,0,0,59
bweeks@fews.net,36,0,0,0,0,0,36
cnakalem@umd.edu,0,13,10,0,1,0,24
daniel.p.sarmiento@nasa.gov,0,36,7,3,0,0,46
eutzschn@umd.edu,0,24,2,2,0,0,28
hkerner@umd.edu,61,19,0,0,0,0,80
izvonkov@umd.edu,9,0,0,0,0,0,9
logdaye@gmail.com,672,48,2,0,0,0,722
meredith.g.brown@nasa.gov,0,8,2,1,0,1,12


In [34]:
# Find total number of labels each person had that ended up being crop_prob of 0.5

user1 = (tigray_halfprob[f'User 1'].value_counts()).to_frame()
halfnamesDf = user1.merge(userDf, left_index=True,right_index=True)

for i in range(2,7):
    useri = (tigray_halfprob[f'User {i}'].value_counts()).to_frame()
    halfnamesDf = halfnamesDf.merge(useri, how='outer',left_index=True,right_index=True)
    
halfnamesDf = halfnamesDf.replace(np.nan, 0)
halfnamesDf = halfnamesDf.astype(int)

halfnamesDf['halfprob_labels'] = 0  
for i in range(1,7):
    halfnamesDf['halfprob_labels'] += halfnamesDf[f'User {i}']
    
halfnamesDf  

,User 1,User 2,User 3,User 4,User 5,User 6,halfprob_labels
alopez11@umd.edu,20,5,0,0,0,0,25
bdubayah@gmail.com,0,11,8,1,0,0,20
bweeks@fews.net,12,0,0,0,0,0,12
cnakalem@umd.edu,0,1,3,0,0,0,4
daniel.p.sarmiento@nasa.gov,0,8,0,0,0,0,8
eutzschn@umd.edu,0,3,1,0,0,0,4
hkerner@umd.edu,14,0,0,0,0,0,14
izvonkov@umd.edu,2,0,0,0,0,0,2
logdaye@gmail.com,120,11,1,0,0,0,132
meredith.g.brown@nasa.gov,0,2,0,0,0,0,2


In [37]:
namesDf['halfprob_labels'] = halfnamesDf['halfprob_labels']
namesDf['percent_halfprob'] = namesDf['halfprob_labels']/namesDf['total_labels']

In [38]:
namesDf

,User 1,User 2,User 3,User 4,User 5,User 6,total_labels,halfprob_labels,percent_halfprob
alopez11@umd.edu,107,40,1,0,0,0,148,25,0.168919
bdubayah@gmail.com,1,29,26,3,0,0,59,20,0.338983
bweeks@fews.net,36,0,0,0,0,0,36,12,0.333333
cnakalem@umd.edu,0,13,10,0,1,0,24,4,0.166667
daniel.p.sarmiento@nasa.gov,0,36,7,3,0,0,46,8,0.173913
eutzschn@umd.edu,0,24,2,2,0,0,28,4,0.142857
hkerner@umd.edu,61,19,0,0,0,0,80,14,0.175000
izvonkov@umd.edu,9,0,0,0,0,0,9,2,0.222222
logdaye@gmail.com,672,48,2,0,0,0,722,132,0.182825
meredith.g.brown@nasa.gov,0,8,2,1,0,1,12,2,0.166667
